# SyncFile2CSV Usage Instructions

This notebook converts .epi7 sync files to CSV format and merges them.

## Prerequisites
```bash
pip install cryptography python-dotenv
```

## Setup
Ensure `.env` file exists in the ipynb folder with:
```
EPI_PASSWORD=your_password
EPI_IV=your_iv
EPI_SALT=your_salt
```

In [1]:
import os, glob, shutil
import pandas as pd
import numpy as np
import re
from fpdf import FPDF
from datetime import datetime
from dotenv import load_dotenv
from syncfile2csv import convert_folder

load_dotenv()
password = os.getenv('EPI_PASSWORD')

change country between: PR, DR, CU

In [2]:
user = "Chris"
country = "PR"

In [3]:
CU_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Cuba"
DR_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom"
PR_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico"

if user == "Chris":
    if country == "DR":
        current_directory = DR_directory
        path1 = DR_directory+"/All_CSVs" #where all CSVs are located
        path2 = DR_directory+"/Sync_Files/2025_10_30/CSV/" #change to show latest upload
        imagepath = DR_directory+"/All_Image_Files"
        audiopath = DR_directory+"/All_Audio_Files"
        country_full = "Dominican Republic"
        epi_info_path = "documents/cadas/data/CADAS data upload/Rep Dom"
    elif country == "CU":
        current_directory = CU_directory
        path1 = CU_directory+"/All_CSVs" #where we merge all cumulative CSV files into one file
        imagepath = CU_directory+"/All_Image_Files"
        audiopath = CU_directory+"/All_Audio_Files"
        path2 = CU_directory+"/Sync_Files/2025_09_18/CSV/" #where we check to see that the latest batch of data includes all files it should
        country_full = "Cuba"
        epi_info_path = "documents/cadas/data/CADAS data upload/Cuba"
    elif country == "PR":
        current_directory = PR_directory
        path1 = PR_directory+"/All_CSVs"
        path2 = PR_directory+"/Sync_Files/2026_1_16/CSV/"
        imagepath = PR_directory+"/All_Image_Files"
        audiopath = PR_directory+"/All_Audio_Files"
        country_full = "Puerto Rico"
        epi_info_path = "documents/cadas/data/CADAS data upload/Puerto_Rico"
elif user == "Ty":
    current_directory = r"C:\Users\Ty\Desktop\DataExport"
else:
    print("Unknown user. Cannot set directory.")
    current_directory = None

if current_directory:
    os.chdir(current_directory)
    cropped_images_path= "/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Pentagons_for_consensus_website"
    file_list = os.listdir(current_directory)
    print(f"Changed directory to: {current_directory}")
    print(cropped_images_path)
    print(path1)
    print(path2)
    print(file_list)
    
imagenames = os.listdir(imagepath)
audionames = os.listdir(audiopath)

path3 = path2+"../EXCEL/"
os.makedirs(path3, exist_ok=True)
print(path3)

incoming_SYNC = path2 + '../SYNC'

Changed directory to: /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico
/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Pentagons_for_consensus_website
/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs
/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/Sync_Files/2026_1_16/CSV/
['CADAS Tracking Sheet.gsheet', 'CSV_Merged', '.DS_Store', 'case_count_report.pdf', 'PR CADAS Tracking Sheet (Copy).gsheet', 'Clusters', 'credentiales de nuve.gsheet', 'All_Audio_Files', 'Resumen', 'Figures', 'PR lab results', 'PR_CADAS_Interviewer_List.gsheet', 'All_Image_Files', 'Sync_files', 'All_CSVs']
/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/Sync_Files/2026_1_16/CSV/../EXCEL/


converting the sync files to csv and copying to path1

In [4]:
# Convert .epi7 sync files to CSV and copy to path1 and path2
convert_folder(incoming_SYNC, password)

incoming_CSV_path = incoming_SYNC + '/CSV_convert'
for csv_file in os.listdir(incoming_CSV_path):
    if csv_file.endswith('.csv'):
        src = os.path.join(incoming_CSV_path, csv_file)
        dst1 = os.path.join(path1, csv_file)
        dst2 = os.path.join(path2, csv_file)
        shutil.copy(src, dst1)
        shutil.copy(src, dst2)
        print(f"Copied to path1 and path2: {csv_file}")

# Delete the CSV_convert folder
shutil.rmtree(incoming_CSV_path)
print(f"Deleted: {incoming_CSV_path}")

Converted: Door_202601161017_1_PR18.epi7
Converted: Cognitive_Scoring_202601161017_8_PR18.epi7
Converted: Sample_Contact_Investigation_202508291156_0_PR18.epi7
Converted: Door_202508291156_26_PR18.epi7
Converted: Listas_202601161012_0_PR19.epi7
Converted: Familiar_202601161016_6_PR18.epi7
Converted: Door_202601161017_13_PR18.epi7
Converted: _participants_202601161016_5_PR18.epi7
Converted: _informationdoor_202601161017_40_PR18.epi7
Converted: _physical_exam_202601161017_1_PR18.epi7
Converted: _sociodemographic_202508291156_2_PR18.epi7
Converted: _cognitive_202508291156_1_PR18.epi7
Converted: Door_202508291156_32_PR18.epi7
Converted: _informationdoorparticipants_202601161016_7_PR18.epi7
Converted: Informante_202601161016_3_PR18.epi7
Converted: Door_202601161012_1_PR19.epi7
Converted: Cognitive_Scoring_202508291156_1_PR18.epi7
Converted: Cognitiva_202601161016_3_PR18.epi7
Converted: Door_202601161017_26_PR18.epi7
Converted: Door_202508291156_8_PR18.epi7
Converted: Door_202508291156_13_PR

here, we're checking to see if there's the same amount of objects in he incoming sync files compared to after they're processed.

In [5]:
incoming_SYNC_files = os.listdir(path2 + '../SYNC')
incoming_CSV_files = os.listdir(path2)

incoming_SYNC_names = [s.split('.')[0] for s in incoming_SYNC_files if s.endswith('.epi7')]
incoming_CSV_names = [s.split('.')[0] for s in incoming_CSV_files if s.endswith('.csv')]

if len(incoming_SYNC_names) != len(incoming_CSV_names):
    print(f"Discrepancy in number of CSV compared to sync files. There are {len(incoming_SYNC_names)} in sync but {len(incoming_CSV_names)} in CSV.")
else:
    print("Correct number of CSV compared to sync files.")
    
for sync_file in incoming_SYNC_names:
    if sync_file not in incoming_CSV_names:
        print(f"File {sync_file} is not present in incoming_CSV")

Correct number of CSV compared to sync files.


In [6]:
cropped_image_list = os.listdir(cropped_images_path)
cropped_image_list = [image for image in cropped_image_list if "(" not in image]

Received files document

In [7]:
# Get the list of all files in the latest file upload
dir_list1 = os.listdir(path2)

# To see which files I received from the data manager (for now not doing anything with this)
newdir_list1 = [x for x in dir_list1 if x.endswith('.epi7')]

# To see which files I was able to process with Epi Info tool
dir_list2 = os.listdir(path2)
newdir_list2 = [x for x in dir_list2 if x.endswith('.csv')]

def remove_common(a, b):
    for i in a[:]:
        if i in b:
            a.remove(i)
            b.remove(i)

    print("list1 : ", a)
    print("list2 : ", b)

remove_common(newdir_list1, newdir_list2)

words_to_filter = ["Neighborhood", "Door", "_informationdoor_", "_informationdoorparticipants", "Listas", "_mainhousehold", "_participants", "_rosterhousehold", "_nonresidentchildren", "Cognitva", "_cognitive", "Cognitive_Scoring", "Examen_Físico", "_physical_exam","Familiar", "_household", "Informante", "_informant", "_sociodem"]

# breaking the longer list into smaller lists
filtered_lists = {word: [] for word in words_to_filter}

for text in newdir_list2:
    for word in words_to_filter:
        if word in text:
            filtered_lists[word].append(text)

# change this number depending on how many tablets worth of information you're receiving
for word, filtered_list in filtered_lists.items():
    while len(filtered_list) < 27:
        filtered_list.append("")

Received_Files = pd.DataFrame()

for word, filtered_list in filtered_lists.items():
    Received_Files[word] = pd.Series(filtered_list)

Received_Files.to_excel(path2+'/Received_Files_Checklist.xlsx', index=False)

Received_Files

list1 :  []
list2 :  ['Sangre_202601160937_0_PR10.csv', '_physical_exam_202601161017_2_PR18.csv', '_physical_exam_202601161017_3_PR18.csv', '_physical_exam_202508291156_2_PR18.csv', '_physical_exam_202508291156_3_PR18.csv', '_informationdoorparticipants_202601161016_2_PR18.csv', '_informationdoorparticipants_202601161016_3_PR18.csv', '_informationdoor_202601161017_40_PR18.csv', '_informationdoor_202601161017_41_PR18.csv', 'Cognitiva_202508291155_4_PR18.csv', 'Cognitiva_202508291155_5_PR18.csv', '_informationdoor_202508291156_35_PR18.csv', '_informationdoor_202508291156_34_PR18.csv', 'Door_202601161017_12_PR18.csv', 'Door_202601161017_13_PR18.csv', '_rosterhousehold_202601161012_0_PR19.csv', 'Listas_202601161017_6_PR18.csv', 'Cognitive_Scoring_202601161017_8_PR18.csv', 'Door_202508291156_10_PR18.csv', 'Door_202508291156_11_PR18.csv', '_mainhousehold_202601161017_3_PR18.csv', '_mainhousehold_202601161017_2_PR18.csv', 'Familiar_202508291155_4_PR18.csv', '_informationdoor_202601161017_37_P

,Neighborhood,Door,_informationdoor_,_informationdoorparticipants,Listas,_mainhousehold,_participants,_rosterhousehold,_nonresidentchildren,Cognitva,_cognitive,Cognitive_Scoring,Examen_Físico,_physical_exam,Familiar,_household,Informante,_informant,_sociodem
0,Neighborhood_202508291155_0_PR18.csv,Door_202601161017_12_PR18.csv,_informationdoor_202601161017_40_PR18.csv,_informationdoorparticipants_202601161016_2_PR...,Listas_202601161017_6_PR18.csv,_mainhousehold_202601161017_3_PR18.csv,_participants_202601161016_5_PR18.csv,_rosterhousehold_202601161012_0_PR19.csv,_nonresidentchildren_202508291155_7_PR18.csv,,_cognitive_202508291156_4_PR18.csv,Cognitive_Scoring_202601161017_8_PR18.csv,Examen_Físico_202601161012_0_PR19.csv,_physical_exam_202601161017_2_PR18.csv,Familiar_202508291155_4_PR18.csv,_household_202601161017_6_PR18.csv,Informante_202601161016_1_PR18.csv,_informant_202601161017_3_PR18.csv,_sociodemographic_202601161017_5_PR18.csv
1,Neighborhood_202601161017_0_PR18.csv,Door_202601161017_13_PR18.csv,_informationdoor_202601161017_41_PR18.csv,_informationdoorparticipants_202601161016_3_PR...,Listas_202601160937_0_PR10.csv,_mainhousehold_202601161017_2_PR18.csv,_participants_202601161016_4_PR18.csv,_rosterhousehold_202601161016_2_PR18.csv,_nonresidentchildren_202508291155_6_PR18.csv,,_cognitive_202601161017_4_PR18.csv,Cognitive_Scoring_202601161017_3_PR18.csv,Examen_Físico_202601161017_0_PR18.csv,_physical_exam_202601161017_3_PR18.csv,Familiar_202601161016_2_PR18.csv,_household_202601160937_0_PR10.csv,Informante_202601161016_0_PR18.csv,_informant_202601161017_2_PR18.csv,_sociodemographic_202601161017_4_PR18.csv
2,,Door_202508291156_10_PR18.csv,_informationdoor_202508291156_35_PR18.csv,_informationdoorparticipants_202601161016_8_PR...,Listas_202601161012_0_PR19.csv,_mainhousehold_202508291155_0_PR18.csv,_participants_202508291155_3_PR18.csv,_rosterhousehold_202601160937_0_PR10.csv,_nonresidentchildren_202601161016_1_PR18.csv,,_cognitive_202601161017_5_PR18.csv,Cognitive_Scoring_202601161017_2_PR18.csv,Examen_Físico_202601161017_1_PR18.csv,_physical_exam_202508291156_2_PR18.csv,Familiar_202601161016_3_PR18.csv,_household_202508291155_3_PR18.csv,Informante_202601161011_0_PR19.csv,_informant_202601161011_0_PR19.csv,_sociodemographic_202601161017_2_PR18.csv
3,,Door_202508291156_11_PR18.csv,_informationdoor_202508291156_34_PR18.csv,_informationdoorparticipants_202601161016_9_PR...,Listas_202508291156_0_PR18.csv,_mainhousehold_202508291155_1_PR18.csv,_participants_202508291155_2_PR18.csv,_rosterhousehold_202601161016_0_PR18.csv,_nonresidentchildren_202601161016_0_PR18.csv,,_cognitive_202508291156_3_PR18.csv,Cognitive_Scoring_202508291156_3_PR18.csv,Examen_Físico_202508291156_0_PR18.csv,_physical_exam_202508291156_3_PR18.csv,Familiar_202601161016_5_PR18.csv,_household_202508291155_2_PR18.csv,Informante_202601161016_6_PR18.csv,_informant_202508291155_0_PR18.csv,_sociodemographic_202601161017_3_PR18.csv
4,,Door_202601161017_18_PR18.csv,_informationdoor_202601161017_37_PR18.csv,_informationdoorparticipants_202508291155_4_PR...,Listas_202508291156_1_PR18.csv,_mainhousehold_202601161011_0_PR19.csv,_participants_202601160937_0_PR10.csv,_rosterhousehold_202601161016_1_PR18.csv,_nonresidentchildren_202601161016_6_PR18.csv,,_cognitive_202508291156_2_PR18.csv,Cognitive_Scoring_202508291156_2_PR18.csv,Examen_Físico_202508291156_1_PR18.csv,_physical_exam_202508291155_0_PR18.csv,Familiar_202601161016_4_PR18.csv,_household_202601161017_0_PR18.csv,Informante_202508291155_0_PR18.csv,_informant_202508291155_1_PR18.csv,_sociodemographic_202508291156_2_PR18.csv
5,,Door_202601161017_19_PR18.csv,_informationdoor_202601161017_36_PR18.csv,_informationdoorparticipants_202508291155_5_PR...,Listas_202601161017_0_PR18.csv,_mainhousehold_202601161017_4_PR18.csv,_participants_202508291155_4_PR18.csv,_rosterhousehold_202508291155_1_PR18.csv,_nonresidentchildren_202601161016_7_PR18.csv,,_cognitive_202601161017_3_PR18.csv,Cognitive_Scoring_202601161017_4_PR18.

In [8]:
if user == "Chris":
    if country == "CU":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/Cuba/All_CSVs/"
    elif country == "DR":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/Rep \Dom/All_CSVs/"
    elif country == "PR":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/"
elif user == "Ty":
    if country == "CU":
        pattern = r"C:\Users\Ty\Desktop\DataExport/"
    elif country == "DR":
        pattern = r"C:\Users\Ty\Desktop\DataExport/"
else:
    print("Unknown user. Cannot set directory.")
    current_directory = None

folder_path = path1 #goes through this whole folder

####COGNITIVE CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_cognitive*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + "_cognitive_"
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_child = pd.concat(dfs, ignore_index=True)

cog_child['num_missing'] = cog_child.isna().sum(axis=1)

# Adjust 'num_missing' so that rows with fewer missing values have higher numbers
cog_child['num_missing'] = cog_child['num_missing'].max() - cog_child['num_missing']

# Sort the DataFrame by 'num_missing' in descending order
cog_child = cog_child.sort_values(by='num_missing', ascending=False)

# Drop the 'filename' column
cog_child = cog_child.drop(columns=['filename'])

#restoring old images
images = [
    'C_72_1_PIC', 'C_72_2_PIC', 'C_72_3_PIC', 'C_72_4_PIC',
    'C_79_1_PIC', 'C_79_2_PIC', 'C_79_3_PIC', 'C_79_4_PIC',
    'ANIM_PIC', 'SYMB_PIC', 'PENT_PIC'
]

# Calculate image completeness score for each row
cog_child['image_completeness_score'] = cog_child[images].notna().sum(axis=1)

# Sort by GlobalRecordId and image completeness (best data first)
cog_child = cog_child.sort_values(
    by=['GlobalRecordId', 'image_completeness_score'], 
    ascending=[True, False]
)

# Column-wise restoration: for each GlobalRecordId, collect the best available data for each image column
restored_images_list = []

for global_id, group in cog_child.groupby('GlobalRecordId'):
    restored_row = {'GlobalRecordId': global_id}
    
    # For each image column, take the first non-missing value from the sorted group
    for img_col in images:
        non_missing_vals = group.loc[group[img_col].notna(), img_col]
        if not non_missing_vals.empty:
            restored_row[img_col] = non_missing_vals.iloc[0]
        else:
            restored_row[img_col] = np.nan
    
    restored_images_list.append(restored_row)

restored_images_df = pd.DataFrame(restored_images_list)

# Keep only the best row per GlobalRecordId for the main data
cog_child = cog_child.drop_duplicates(subset='GlobalRecordId', keep='first')

# Replace image columns with restored versions
cog_child = cog_child.drop(columns=images + ['image_completeness_score'])
cog_child = cog_child.merge(restored_images_df, on='GlobalRecordId', how='left')

#done restoring images here

pattern_to_remove = r'=HYPERLINK\("media\\([^"]+)",\"<CLICK HERE>"\)'
cog_child = cog_child.replace(pattern_to_remove, r'\1', regex=True)

#TODO: REVISIT THIS IF NOT WORKING AS INTENDED
if country == "CU":
    cog_child['C_72_1_PIC'] = np.nan
    cog_child['C_72_2_PIC'] = np.nan
    cog_child['C_72_3_PIC'] = np.nan
    cog_child['C_79_1_PIC'] = np.nan
    cog_child['C_79_2_PIC'] = np.nan
    cog_child['C_79_3_PIC'] = np.nan

columns_to_check = [
    'C_72_1_PIC', 'C_72_2_PIC', 'C_72_3_PIC', 'C_72_4_PIC',
    'C_79_1_PIC', 'C_79_2_PIC', 'C_79_3_PIC', 'C_79_4_PIC',
    'ANIM_PIC', 'SYMB_PIC', 'PENT_PIC',
    'G_1_File', 'G_2_File', 'G_2_File2', 'G_3_File', 'G_3_File2'
]

for column in columns_to_check:
    foundlist = []
    piclist = cog_child[column].tolist()
    file_list = audionames if 'File' in column else imagenames
    

    for pic in piclist:
        if pic in file_list:
            foundlist.append("found")
        else:
            foundlist.append("missing")

    cog_child[column + '_FOUND'] = foundlist

    missing_df = cog_child[cog_child[f'{column}_FOUND'] == 'missing']

    if not missing_df.empty:
        filename = os.path.join(path3, f"missing_{column}.xlsx")
        missing_df.to_excel(filename, index=False)

# checking for missing filepaths
pics_to_check = [
    'C_72_1_PIC', 'C_72_2_PIC', 'C_72_3_PIC', 'C_72_4_PIC',
    'C_79_1_PIC', 'C_79_2_PIC', 'C_79_3_PIC', 'C_79_4_PIC',
    'PENT_PIC']

attempt_cols = [
    'C_72_1',  'C_72_2',  'C_72_3',  'C_72_4', 
    'C_79_1',  'C_79_2',  'C_79_3',  'C_79_4',
    'C_32']

# First loop: pic missing but attempt = 0 (not attempted)
for pic_col, attempt_col in zip(pics_to_check, attempt_cols):
    condition = cog_child[pic_col].isna() & (cog_child[attempt_col] == 0)
    missing_df = cog_child.loc[condition]
    if not missing_df.empty:
        filename = os.path.join(path3, f"missing_{pic_col}_filepath.xlsx")
        missing_df.to_excel(filename, index=False)

# Second loop: pic missing but attempt = 1 (attempted)
pics_to_check_2 = ['ANIM_PIC', 'SYMB_PIC']
attempt_cols_2 = ['C_40', 'C_43']

for pic_col2, attempt_col2 in zip(pics_to_check_2, attempt_cols_2):
    condition = cog_child[pic_col2].isna() & (cog_child[attempt_col2] == 1)
    missing_df = cog_child.loc[condition]
    if not missing_df.empty:
        filename = os.path.join(path3, f"missing_{pic_col2}_filepath.xlsx")
        missing_df.to_excel(filename, index=False)

cog_child['pent_pic_cropped'] = cog_child['PENT_PIC'].apply(lambda x: 'cropped' if x in cropped_image_list else pd.NA)

for column in cog_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        cog_child[column] = cog_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)
        
cog_child.to_csv("CSV_Merged/Cog_Child.csv")

#exporting the automatic report of cases
case_count = len(cog_child)
unique_globalrecordid_count = cog_child['GlobalRecordId'].nunique()  # Ensure column name matches

# Create PDF object
pdf = FPDF()
pdf.add_page()

pdf.set_font("Helvetica", size=24)
pdf.cell(200, 10, txt="Epi Info", ln=True, align='L')

pdf.set_font("Arial", size=12)
pdf.set_font("Arial", style="I")
pdf.cell(200, 10, txt=f"Current Data Source: \n{epi_info_path}", ln=True, align='L')
pdf.set_font("Arial", style="")
pdf.cell(200, 10, txt=f"Date: {datetime.now().strftime('%m/%d/%Y %I:%M:%S %p')}", ln=True, align='L')
pdf.cell(200, 10, txt=f"Record Count: {case_count} (Deleted Records Included) in {country_full}", ln=True, align='L')

# Add summary section
pdf.ln(10)
pdf.cell(200, 10, txt="SUMMARIZE Cases :: Count ( UniqueKey ) TO Counts", ln=True, align='L')
pdf.ln(5)
pdf.cell(200, 10, txt="Interview", ln=True, align='L')
pdf.cell(200, 10, txt=str(unique_globalrecordid_count), ln=True, align='L')

# Save output
pdf.output("case_count_report.pdf")

####COGNITIVE PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Cognitiva*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Cognitiva_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_parent = pd.concat(dfs, ignore_index=True)
cog_parent = cog_parent.sort_values(by='filename', ascending=False)
cog_parent = cog_parent.drop(columns=['filename'])
cog_parent = cog_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_parent.to_csv("CSV_Merged/Cog_Parent.csv")

def find_orphaned_images(cog_child, image_columns, imagepath, output_path):
    """
    Find images in imagepath directory that are not referenced in any dataframe columns.
    """
    # Get all image files from the directory
    image_files_in_path = set(os.listdir(imagepath))
    
    # Collect all unique image references from all columns
    images_referenced = set()
    for col in image_columns:
        # Add non-null values to the set
        images_referenced.update(cog_child[col].dropna().unique())
    
    # Find orphaned images (in directory but not referenced)
    orphaned_images = list(image_files_in_path - images_referenced)
    
    # Export to Excel if any orphaned images found
    if orphaned_images:
        df_orphaned = pd.DataFrame(orphaned_images, columns=['OrphanedImages'])
        filename = os.path.join(output_path, "orphaned_images.xlsx")
        df_orphaned.to_excel(filename, index=False)
        print(f"Found {len(orphaned_images)} orphaned images, exported to {filename}")
    else:
        print("No orphaned images found")

# Usage with your image columns
all_image_columns = [
    'C_72_1_PIC', 'C_72_2_PIC', 'C_72_3_PIC', 'C_72_4_PIC',
    'C_79_1_PIC', 'C_79_2_PIC', 'C_79_3_PIC', 'C_79_4_PIC',
    'ANIM_PIC', 'SYMB_PIC', 'PENT_PIC'
]

find_orphaned_images(cog_child, all_image_columns, imagepath, path3)

####COGNITIVE SCORING#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Cognitive_Scoring*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Cognitive_Scoring_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_scoring = pd.concat(dfs, ignore_index=True)
cog_scoring['num_missing'] = cog_scoring.isna().sum(axis=1)
cog_scoring['num_missing'] = cog_scoring['num_missing'].max() - cog_scoring['num_missing']

cog_scoring = cog_scoring.sort_values(by='filename', ascending=False)

cog_scoring = cog_scoring.drop(columns=['filename','num_missing'])

cog_scoring = cog_scoring.drop_duplicates(subset='GlobalRecordId', keep='first')

for column in cog_scoring.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        cog_scoring[column] = cog_scoring[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)

cog_scoring.to_csv("CSV_Merged/Cog_Scoring.csv")

####HOUSEHOLD CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_household*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_household_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
household_child = pd.concat(dfs, ignore_index=True)
household_child['num_missing'] = household_child.isna().sum(axis=1)
household_child['num_missing'] = household_child['num_missing'].max() - household_child['num_missing']

household_child = household_child.sort_values(by='filename', ascending=False)

household_child = household_child.drop_duplicates(subset='GlobalRecordId', keep='first')

household_child = household_child.drop(columns=['filename','num_missing'])


for column in household_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        household_child[column] = household_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)


household_child.to_csv("CSV_Merged/Household_Child.csv")

####HOUSEHOLD PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Familiar*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Familiar_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
household_parent = pd.concat(dfs, ignore_index=True)
household_parent = household_parent.sort_values(by='filename', ascending=False)
household_parent = household_parent.drop(columns=['filename'])
household_parent = household_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

household_parent.to_csv("CSV_Merged/Household_Parent.csv")

####PHYS CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_physical_exam*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_physical_exam_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
phys_child = pd.concat(dfs, ignore_index=True)
phys_child['num_missing'] = phys_child.isna().sum(axis=1)
phys_child['num_missing'] = phys_child['num_missing'].max() - phys_child['num_missing']

phys_child = phys_child.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

phys_child = phys_child.drop(columns=['filename','num_missing'])

phys_child = phys_child.drop_duplicates(subset='GlobalRecordId', keep='first')

for column in phys_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        phys_child[column] = phys_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)


phys_child.to_csv("CSV_Merged/Phys_Child.csv")

####PHYS PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Examen_Físico*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Examen_Físico_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
phys_parent = pd.concat(dfs, ignore_index=True)
phys_parent = phys_parent.sort_values(by='filename', ascending=False)
phys_parent = phys_parent.drop(columns=['filename'])
phys_parent = phys_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

phys_parent.to_csv("CSV_Merged/Phys_Parent.csv")

####INFOR CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informant*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informant_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
infor_child = pd.concat(dfs, ignore_index=True)
infor_child['num_missing'] = infor_child.isna().sum(axis=1)
infor_child['num_missing'] = infor_child['num_missing'].max() - infor_child['num_missing']

infor_child = infor_child.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

infor_child = infor_child.drop(columns=['filename','num_missing'])
infor_child = infor_child.drop_duplicates(subset='GlobalRecordId', keep='first')

for column in infor_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        infor_child[column] = infor_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)


infor_child.to_csv("CSV_Merged/Infor_Child.csv")

####INFOR PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Informante*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Informante_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
infor_parent = pd.concat(dfs, ignore_index=True)
infor_parent = infor_parent.sort_values(by='filename', ascending=False)
infor_parent = infor_parent.drop(columns=['filename'])
infor_parent = infor_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

infor_parent.to_csv("CSV_Merged/Infor_Parent.csv")

####SOCIO CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_sociodemographic*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_sociodemographic_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
socio_child = pd.concat(dfs, ignore_index=True)
socio_child['num_missing'] = socio_child.isna().sum(axis=1)
socio_child['num_missing'] = socio_child['num_missing'].max() - socio_child['num_missing']

socio_child = socio_child.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

socio_child = socio_child.drop(columns=['filename','num_missing'])
socio_child = socio_child.drop_duplicates(subset='GlobalRecordId', keep='first')

for column in socio_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        socio_child[column] = socio_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)


socio_child.to_csv("CSV_Merged/Socio_Child.csv")

####SOCIO PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Sociodemográfica*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Sociodemográfica_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
socio_parent = pd.concat(dfs, ignore_index=True)
socio_parent = socio_parent.sort_values(by='filename', ascending=False)
socio_parent = socio_parent.drop(columns=['filename'])
socio_parent = socio_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

socio_parent.to_csv("CSV_Merged/Socio_Parent.csv")

####NEIGHBORHOOD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Neighborhood*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Neighborhood_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
neighborhood = pd.concat(dfs, ignore_index=True)
neighborhood['num_missing'] = neighborhood.isna().sum(axis=1)
neighborhood['num_missing'] = neighborhood['num_missing'].max() - neighborhood['num_missing']

neighborhood = neighborhood.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

neighborhood = neighborhood.drop(columns=['filename','num_missing'])
neighborhood = neighborhood.drop_duplicates(subset='GlobalRecordId', keep='first')

neighborhood.to_csv("CSV_Merged/Neighborhood.csv")

####DOOR PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Door*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Door_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
door_parent = pd.concat(dfs, ignore_index=True)
door_parent = door_parent.sort_values(by='filename', ascending=False)
door_parent = door_parent.drop(columns=['filename'])
door_parent = door_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

door_parent.to_csv("CSV_Merged/Door.csv")

####DOOR CHILD 1#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informationdoor_*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informationdoor_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
informationdoor = pd.concat(dfs, ignore_index=True)

informationdoor['D_HOUSEID'] = informationdoor.groupby('GlobalRecordId')['D_HOUSEID'].transform(lambda x: x.ffill().bfill())
informationdoor['D_CLUSTID'] = informationdoor.groupby('GlobalRecordId')['D_CLUSTID'].transform(lambda x: x.ffill().bfill())
informationdoor['D_INTERID'] = informationdoor.groupby('GlobalRecordId')['D_INTERID'].transform(lambda x: x.ffill().bfill())

informationdoor['num_missing'] = informationdoor.isna().sum(axis=1)
informationdoor['num_missing'] = informationdoor['num_missing'].max() - informationdoor['num_missing']

informationdoor = informationdoor.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

informationdoor = informationdoor.drop(columns=['filename','num_missing'])

informationdoor = informationdoor.drop_duplicates(subset='GlobalRecordId', keep='first')

informationdoor.to_csv("CSV_Merged/InformationDoor.csv")

####DOOR CHILD 2#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informationdoorparticipants*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informationdoorparticipants_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
informationdoorparticipants = pd.concat(dfs, ignore_index=True)

informationdoorparticipants['num_missing'] = informationdoorparticipants.isna().sum(axis=1)
informationdoorparticipants['num_missing'] = informationdoorparticipants['num_missing'].max() - informationdoorparticipants['num_missing']

informationdoorparticipants = informationdoorparticipants.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

informationdoorparticipants = informationdoorparticipants.drop(columns=['filename','num_missing'])

informationdoorparticipants = informationdoorparticipants.drop_duplicates(subset='GlobalRecordId', keep='first')

informationdoorparticipants.to_csv("CSV_Merged/InformationDoorParticipants.csv")

####ROSTER PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Listas*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Listas_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
rosters_parent = pd.concat(dfs, ignore_index=True)
rosters_parent = rosters_parent.sort_values(by='filename', ascending=False)
rosters_parent = rosters_parent.drop(columns=['filename'])
rosters_parent = rosters_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

rosters_parent.to_csv("CSV_Merged/Roster_Parent.csv")

####ROSTER CHILD 1#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_mainhousehold*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_mainhousehold_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
mainhousehold = pd.concat(dfs, ignore_index=True)

mainhousehold['num_missing'] = mainhousehold.isna().sum(axis=1)
mainhousehold['num_missing'] = mainhousehold['num_missing'].max() - mainhousehold['num_missing']

mainhousehold = mainhousehold.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

mainhousehold = mainhousehold.drop(columns=['filename','num_missing'])
mainhousehold = mainhousehold.drop_duplicates(subset='GlobalRecordId', keep='first')

mainhousehold.to_csv("CSV_Merged/MainHousehold.csv")

####ROSTER CHILD 2#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_participants*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_participants_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
participants = pd.concat(dfs, ignore_index=True)

participants['num_missing'] = participants.isna().sum(axis=1)
participants['num_missing'] = participants['num_missing'].max() - participants['num_missing']

participants = participants.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

participants = participants.drop(columns=['filename','num_missing'])
participants = participants.drop_duplicates(subset='GlobalRecordId', keep='first')

participants.to_csv("CSV_Merged/Participants.csv")

####ROSTER CHILD 3#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_rosterhousehold*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_rosterhousehold_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
nonparticipants = pd.concat(dfs, ignore_index=True)

nonparticipants['num_missing'] = nonparticipants.isna().sum(axis=1)
nonparticipants['num_missing'] = nonparticipants['num_missing'].max() - nonparticipants['num_missing']

nonparticipants = nonparticipants.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

nonparticipants = nonparticipants.drop(columns=['filename','num_missing'])

nonparticipants = nonparticipants.drop_duplicates(subset='GlobalRecordId', keep='first')

nonparticipants.to_csv("CSV_Merged/NonParticipants.csv")

####ROSTER CHILD 4#######

dfs = [] 

for filename in glob.glob(os.path.join(folder_path, '*_nonresidentchildren*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_nonresidentchildren_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
nonresidentchildren = pd.concat(dfs, ignore_index=True)

nonresidentchildren['num_missing'] = nonresidentchildren.isna().sum(axis=1)
nonresidentchildren['num_missing'] = nonresidentchildren['num_missing'].max() - nonresidentchildren['num_missing']

nonresidentchildren = nonresidentchildren.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

nonresidentchildren = nonresidentchildren.drop(columns=['filename','num_missing'])

nonresidentchildren = nonresidentchildren.drop_duplicates(subset='GlobalRecordId', keep='first')

nonresidentchildren.to_csv("CSV_Merged/NonResidentChildren.csv")

if country != "CU":
    dfs = []  # list to hold all dataframes
    for filename in glob.glob(os.path.join(folder_path, '*Sangre*.csv')):
        print(f"Processing {filename}")
        df = pd.read_csv(filename)
        df['filename'] = filename
        pattern_to_remove = pattern + 'Sangre_'
        df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
        df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
        df['filename'] = df['filename'].astype(int)
        dfs.append(df)
        
    sangre = pd.concat(dfs, ignore_index=True)
    sangre = sangre.sort_values(by='filename', ascending=False)
    sangre = sangre.drop(columns=['filename'])
    sangre = sangre.drop_duplicates(subset='GlobalRecordId', keep='first')
    
    sangre.to_csv("CSV_Merged/Sangre.csv")

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202507031127_0_PR18.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202406050824_0_PR08.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202508110236_0_PR19.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202412030228_0_PR10.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202403221159_0_PR_3.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202508270938_0_PR20.cs

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/1133271506.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cog_child['num_missing'] = cog_child.isna().sum(axis=1)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/1133271506.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cog_child['image_completeness_score'] = cog_child[images].notna().sum(axis=1)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/1133271506.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is 

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202404050052_0_PR03.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202501220256_0_PR12.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202412020414_0_PR02.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202508291155_4_PR18.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202508291155_5_PR18.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202508130258_0_28.csv
Proces

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/1133271506.py:300: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  household_child['num_missing'] = household_child.isna().sum(axis=1)


Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Familiar_202412020112_0_PR07.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Familiar_202404050052_0_PR03.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Familiar_202501220256_0_PR12.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Familiar_202412020414_0_PR02.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Familiar_202508291155_4_PR18.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Familiar_202503311148_0_PR_00.csv
Processin

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/1133271506.py:411: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  infor_child['num_missing'] = infor_child.isna().sum(axis=1)


Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202501220257_0_PR12.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202508130258_1_28.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202505020002_0_PR10.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202507290033_0_PR18.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202507290033_1_PR18.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202112210411_0_a2838941b

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/1133271506.py:465: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  socio_child['num_missing'] = socio_child.isna().sum(axis=1)


Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202505261005_0_PR14.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202508140300_0_22.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202508130258_2_28.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202505260954_0_PR20.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202508140313_0_PR07.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/So

adding original PID's as received from the tablets

In [9]:
datasets_full = ['cog_child', 'phys_child', 'infor_child', 'socio_child']

# Process datasets that need both pid and hhid
for dataset_name in datasets_full:
    df = globals()[dataset_name]  # or however you access your datasets
    
    # Get the first letter of the dataset name for the prefix, capitalized
    prefix = dataset_name[0].upper()
    
    # Convert numeric columns to strings with formatting (handle NaN values)
    df[f'{prefix}_country_str'] = df[f'{prefix}_Country'].fillna(0).astype(int).astype(str)
    df[f'{prefix}_clustid_str'] = df[f'{prefix}_CLUSTID'].fillna(0).astype(int).astype(str).str.zfill(2)
    df[f'{prefix}_houseid_str'] = df[f'{prefix}_HOUSEID'].fillna(0).astype(int).astype(str).str.zfill(3)
    df[f'{prefix}_particid_str'] = df[f'{prefix}_PARTICID'].fillna(0).astype(int).astype(str).str.zfill(2)
    
    # Create concatenated IDs
    df['pid_original'] = df[f'{prefix}_country_str'] + df[f'{prefix}_clustid_str'] + df[f'{prefix}_houseid_str'] + df[f'{prefix}_particid_str']
    df['hhid_original'] = df[f'{prefix}_country_str'] + df[f'{prefix}_clustid_str'] + df[f'{prefix}_houseid_str']
    
    # Drop temporary string columns
    df.drop(columns=[f'{prefix}_country_str', f'{prefix}_clustid_str', f'{prefix}_houseid_str', f'{prefix}_particid_str'], inplace=True)
    df.to_csv(f"CSV_Merged/{dataset_name.replace('_', '_').title().replace('_', '_')}.csv", index=False)

# Process household_child dataset (only hhid)
df = household_child
df['H_country_str'] = df['H_Country'].fillna(0).astype(int).astype(str)
df['H_clustid_str'] = df['H_CLUSTID'].fillna(0).astype(int).astype(str).str.zfill(2)
df['H_houseid_str'] = df['H_HOUSEID'].fillna(0).astype(int).astype(str).str.zfill(3)

df['hhid_original'] = df['H_country_str'] + df['H_clustid_str'] + df['H_houseid_str']

df.drop(columns=['H_country_str', 'H_clustid_str', 'H_houseid_str'], inplace=True)
household_child.to_csv("CSV_Merged/Household_Child.csv", index=False)

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/507976163.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{prefix}_country_str'] = df[f'{prefix}_Country'].fillna(0).astype(int).astype(str)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/507976163.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{prefix}_clustid_str'] = df[f'{prefix}_CLUSTID'].fillna(0).astype(int).astype(str).str.zfill(2)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_7175/507976163.py:13: Performanc

In [10]:
# Copy CSV_Merged files to country-specific folder
if country == "DR":
    output_folder = "/Users/chrissoria/Documents/cadas/data/DR_in"
elif country == "PR":
    output_folder = "/Users/chrissoria/Documents/cadas/data/PR_in"
elif country == "CU":
    output_folder = "/Users/chrissoria/Documents/cadas/data/CUBA_in"

os.makedirs(output_folder, exist_ok=True)

csv_merged_path = "CSV_Merged"
for csv_file in os.listdir(csv_merged_path):
    if csv_file.endswith('.csv'):
        src = os.path.join(csv_merged_path, csv_file)
        dst = os.path.join(output_folder, csv_file)
        shutil.copy(src, dst)
        print(f"Copied to {output_folder}: {csv_file}")

print(f"\nAll CSV_Merged files copied to {output_folder}")

Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Cog_Parent.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Cog_Child.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Socio_Child.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Participants.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Household_Child.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Cog_Scoring.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Sangre.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: InformationDoor.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Phys_Parent.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: MainHousehold.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Door.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: Roster_Parent.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: NonResidentChildren.csv
Copied to /Users/chrissoria/Documents/cadas/data/PR_in: In